In [1]:
import torch
import numpy as np
from engine.torchengine import AnalyticalSetSympy, EliminateAnalysisMergeResiduals, EliminateAnalysis, FunctionSympy, ipoptsolvercon
from engine.torchdata import load_file, load_vals, generate_optim_functions, print_formatted_table, perturb
from engine.loaddata import process_json
import json
# Set the print options
np.set_printoptions(formatter={'float': lambda x: "{:0.2f}".format(x).rstrip('0').rstrip('.')})

In [2]:
data = load_file('testproblems')
problem_id = 0
_, fdata = list(data.items())[problem_id]
polynomials, indices, edges, objective = process_json(fdata)
idxrev = {var.item():key for key,var in indices.items()}

In [128]:
def runproblem(problem_id, xs=None, delta=0.1, seed=42):
    _, fdata = list(data.items())[problem_id]
    polynomials, indices, edges, objective = process_json(fdata)

    sets = {idx:AnalyticalSetSympy(poly, indices=indices).reassign(edges[1][idx][0],  rational=True) for idx,poly in polynomials.items()}
    objective = FunctionSympy(objective, indices)

    analyses = {key: s.analysis for key,s in sets.items()}
    residuals = {key: s.residual for key,s in sets.items()}
    order = list(sorted(analyses.keys()))
    A, B = [], order
    # Merge residuals only
    solvefor = torch.tensor([])
    R = EliminateAnalysisMergeResiduals(functions=[residuals[idx] for idx in B])
    P = EliminateAnalysis([analyses[idx] for idx in A], [objective,R])
    objective_idx,residual_idx,equality_idx,inequality_idx = 0,1,None,None
    optim_indices = P.structure[0]
    x0 = perturb(xs, delta, optim_indices, seed=seed)
    # MAKE SURE TO SET INEQUALITY DIRECTION, it is different for scipy and ipopt
    xguess, obj_function, ineq_function, eq_function, dobj, dineq, deq, hobj = generate_optim_functions(P,
        optim_indices, x0, inequality_direction='negative-null', 
        objective=objective_idx, residuals=residual_idx, equalities=equality_idx)
    bnds_problem = [(None, None) for elt in optim_indices]
    xsol, info,storeiter,elapsed_time = ipoptsolvercon(xguess, obj_function, ineq_function, eq_function, dobj, dineq, deq, bnds_problem)
    return x0, storeiter,elapsed_time,info,optim_indices

In [109]:
problem_id = 3 # (0,6), (1, infeas), (3,23)
n = len(indices)
objs = []
for i in range(23, 24):
    torch.manual_seed(i)
    x0 = torch.rand(n, dtype=torch.float64)
    # x0 = torch.ones(n, dtype=torch.float64)
    xsol, storeiter,elapsed_time,info,_ = runproblem(problem_id, x0, delta=0)
    results = {str(key):xsol[val].item() for key,val in indices.items()}
    print(info['obj_val'], 'Iteration number:', i)
    objs.append(info['obj_val'])

4.6420309235691347e-17 Iteration number: 23


In [111]:
name = 'results_polyid_{}.json'.format(str(problem_id))
json_string = json.dumps(results, indent=4)
with open('../applications/data/{}'.format(name), 'w') as file:
    file.write(json_string)

In [133]:
name = 'results_polyid_{}'.format(str(problem_id))
xs = load_vals(name, indices)

In [134]:
xsol, storeiter,elapsed_time,info,optim_indices = runproblem(problem_id, xs, delta=10, seed=37)

In [139]:
info

{'x': array([1, 1, -0, 1, 1, 5.33, 4, 0, 1733448.23, -0.14, -3.17, 0, -1.67,
        12.25, -1516598.89]),
 'g': array([0, -0, 0, 0, 0, -0, 0, 0, -0, 0]),
 'obj_val': 1.0000052034951856,
 'mult_g': array([-0, 0, -0, -0, 0, 0, 0, 0, -0, 0]),
 'mult_x_L': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'mult_x_U': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'status': 0,
 'status_msg': b'Algorithm terminated successfully at a locally optimal point, satisfying the convergence tolerances (can be specified by options).'}

In [135]:
x0 = perturb(xs, 10, optim_indices, seed=37)
print_formatted_table([xs, x0], indices, idxrev)

x_0      x_1    x_2     x_3     x_4    x_5     x_6     x_7   x_8    x_9     x_10    x_11   x_12    x_13  x_14    
-23.333  1      1       -3.167  -0.323 -1.667  1       -4.5  -0.143 17.523  1       1      27.871  3.937 12.25   
-233.311 -3.584 -12.141 -24.025 -3.414 -10.411 -24.583 64.59 -1.019 279.374 -13.946 10.703 339.771 4.623 -151.794


### Test robustness of the solution

In [120]:
seeds = [59, 45, 22, 20, 28, 57,  2, 42, 27, 37]
iters, elapsed_times = [],[]
for seed in seeds:
    x0 = load_vals(name, indices)
    xsol, storeiter,elapsed_time,info = runproblem(problem_id, x0, delta=0.01, seed=seed)
    iters.append(storeiter)
    elapsed_times.append(elapsed_time)

In [121]:
elapsed_times

[3.424323797225952,
 1.1146697998046875,
 3.6352338790893555,
 1.1356408596038818,
 7.380082130432129,
 9.535183429718018,
 1.466709852218628,
 0.7223019599914551,
 3.9251134395599365,
 18.45064640045166]

In [116]:
elapsed_times

[7.621184587478638,
 3.782424211502075,
 5.565443992614746,
 8.332407474517822,
 4.566037178039551,
 4.883890628814697,
 9.666556358337402,
 5.170992851257324,
 3.8253605365753174,
 3.8964433670043945]